In [3]:
import pandas as pd
import numpy as np
from itertools import combinations
import datetime as dt

In [4]:
def drop_outliers(df, columns, frac = 0.05):
    
    df = df.copy()
    
    # Mark outliers for each column
    for column in columns:
        cut_low, cut_high = df[column].quantile([frac/2, (1-frac/2)])
        df[column + '_keep'] = ((df[column] < cut_high) & (df[column] > cut_low)).astype(int)
        
    # Subset to observations that aren't outliers for any column
    query_str = ' & '.join([x + '_keep == 1' for x in columns])
    
    return df.query(query_str).drop([x + '_keep' for x in columns], axis = 1).copy()

def df_drop_na(df, cols):
    
    return df[~df[cols].isna().any(axis=1)]

## Import Data

In [5]:
price_df       = pd.read_csv('../data/electricity/elc_retail_price_monthly.csv', na_values = 'NM')
sales_df       = pd.read_csv('../data/electricity/elc_retail_sales_monthly.csv', na_values = 'NM')
fossil_cost_df = pd.read_csv('../data/instruments/fossil_fuel_cost_monthly.csv', na_values = ['W', '--', 'NM'])
pcepi_df       = pd.read_csv('../data/other/us_pcepi.csv')
ddd_df         = pd.read_csv('../data/controls/state_monthly_hdd_cdd.csv')

## Clean and Merge Data

In [6]:
# Fix state names
sales_df['State']       = sales_df['State'].apply(lambda x: x.strip().lower()) 
price_df['State']       = price_df['State'].apply(lambda x: x.strip().lower()) 
fossil_cost_df['state'] = fossil_cost_df['state'].apply(lambda x: x.strip().lower()) 

In [7]:
# Fix dates
pcepi_df['month'] = pcepi_df['DATE'].apply(lambda x: x.split('/')[0])
pcepi_df['year']  = pcepi_df['DATE'].apply(lambda x: x[-4:])
ddd_df['month']   = ddd_df['month'].astype(str)
ddd_df['year']    = ddd_df['year'].astype(str)

In [8]:
# Reshape dataframes
sales_melt_df  = pd.melt(sales_df, id_vars = ['State'], value_vars = sales_df.columns[1:],
                         var_name = 'date', value_name = 'load')


price_melt_df  = pd.melt(price_df, id_vars = ['State'], value_vars = price_df.columns[1:],
                         var_name = 'date', value_name = 'price')


fossil_cost_df         = pd.melt(fossil_cost_df, id_vars = ['state', 'fuel'], value_vars = fossil_cost_df.columns[4:],
                         var_name = 'date', value_name = 'cost')
fossil_cost_df         = fossil_cost_df.reset_index(drop = True)
fossil_cost_df['cost'] = fossil_cost_df['cost'].fillna(-1234)
fossil_cost_df         = fossil_cost_df.groupby(['state', 'date', 'fuel'])['cost'].sum().unstack('fuel').reset_index()
fossil_cost_df.columns = (list(fossil_cost_df.columns[:2]) + 
                          [x.strip().replace(' ', '_') + '_cost' for x in fossil_cost_df.columns[2:]])

In [9]:
# Merge electricity and instrument data
data_df = price_melt_df.merge(sales_melt_df, on = ['State', 'date']).rename(columns = {'State': 'state'}).merge(
            fossil_cost_df, on = ['state', 'date'])

In [10]:
# Month abbreviations
month_abrv_dict = dict(zip(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                          range(1,13)))

# Clean up columns
data_df['state'] = data_df['state'].apply(lambda x: x.strip().lower()) 
data_df['month'] = data_df['date'].apply(lambda x: str(month_abrv_dict.get(x.split('-')[0])))
data_df['year']  = data_df['date'].apply(lambda x: str(2000+int(x.split('-')[1])))
data_df['price'] = pd.to_numeric(data_df['price'])/100
data_df['load']  = pd.to_numeric(data_df['load'])

In [11]:
# Merge with pcepi and ddd data
data_df = data_df.merge(pcepi_df, on =['month', 'year']).drop('DATE', axis = 1).merge(
                        ddd_df, on = ['state', 'month', 'year'])

In [12]:
# Fix missing entries in fuel cost
fuel_cost_cols = [x for x in data_df.columns if 'cost' in x]
for fuel_cost_col in fuel_cost_cols:
    data_df[fuel_cost_col] = pd.to_numeric(fossil_cost_df[fuel_cost_col]).apply(
        lambda x: np.nan if (x == -1234 or x == 0) else x)
    
# Adjust for inflation
for price_col in (fuel_cost_cols + ['price']):
    data_df[price_col] = data_df[price_col]*100/data_df['PCEPI']
    
# Fix date columns
data_df['month'] = data_df['month'].astype(int) 
data_df['year']  = data_df['year'].astype(int)  

## Construct Regression Data

In [13]:
# Subset dataframe for relevant variables
em_sample_df = data_df[['state', 'price', 'load', 'month', 'year', 'coal_cost', 'CDD', 'HDD']].copy()

# Drop missing and outliers (CDD and HDD data already cleaned)
em_sample_df[~em_sample_df.isna().any(axis = 1)]
em_sample_df = drop_outliers(em_sample_df, columns = ['price', 'load', 'coal_cost'], frac = 0.01).reset_index(drop = True)

# Add date column
em_sample_df['date'] = em_sample_df.apply(lambda x: '{0}/{1}'.format(x.month, x.year), axis = 1)

print('Preliminary Data Set Observations:  ', len(em_sample_df))

Preliminary Data Set Observations:   818


In [14]:
# Get all unique combinations of (t,s,i,j) => (month, month, state, state)
sample_array = np.array([np.array(comb) for comb in combinations(em_sample_df.index, 2)])

# Split into two dataframes for (t,i) and (s,j)
em_sample_df_1         = em_sample_df.loc[sample_array[:,0]].reset_index(drop = True)
em_sample_df_2         = em_sample_df.loc[sample_array[:,1]].reset_index(drop = True)
em_sample_df_1.columns = [x + '_1' for x in em_sample_df_1.columns] 
em_sample_df_2.columns = [x + '_2' for x in em_sample_df_2.columns] 

# Merge and filter out t == s and i != j
em_concat_df = pd.concat([em_sample_df_1, em_sample_df_2], axis = 1)
em_concat_df = em_concat_df.query('state_1 == state_2 & date_1 != date_2').drop_duplicates().reset_index(drop = True)

# Add reg data columns
em_concat_df['ln_load_rel']    = np.log(np.divide(em_concat_df['load_1'],      em_concat_df['load_2']))
em_concat_df['ln_price_rel']   = np.log(np.divide(em_concat_df['price_1'],     em_concat_df['price_2']))
em_concat_df['ln_coal_rel']    = np.log(np.divide(em_concat_df['coal_cost_1'], em_concat_df['coal_cost_2']))

# Regressions should be done with pairs of data for the same state
reg_data_df = em_concat_df.copy()
reg_data_df['time_diff'] = (reg_data_df
                            .apply(lambda x: 
                                   dt.datetime(x.year_1, x.month_1, 1) - dt.datetime(x.year_2, x.month_2, 1), 
                                   axis = 1)
                            .apply(lambda x: np.round(x.days/30.4, 0)))

print('Regression Data Set Observations:  ', len(reg_data_df))

Regression Data Set Observations:   6817


In [19]:
reg_data_df.to_csv('../data/processed/regression_data.csv', index = False)

## Data Description

In [15]:
data_df.query('coal_cost == coal_cost')[['price', 'load', 'coal_cost', 'CDD', 'HDD']].describe()

,price,load,coal_cost,CDD,HDD
count,848.000000,848.00000,848.000000,848.000000,848.000000
mean,0.118393,2454.96934,46.004220,64.662736,443.199292
std,0.027906,2380.24399,17.683376,134.965191,412.267006
min,0.076601,144.00000,24.038135,0.000000,0.000000
25%,0.098699,790.50000,33.429777,0.000000,48.000000
50%,0.110991,1869.50000,41.396633,0.000000,367.000000
75%,0.128693,3315.00000,51.831937,47.250000,754.500000
max,0.208995,18621.00000,107.489340,761.000000,1794.000000


In [23]:
print(data_df.query('coal_cost == coal_cost')[['price', 'load', 'coal_cost', 'CDD', 'HDD']]
      .describe()
      .drop('count', axis = 0)
      .T
      .round(decimals = 4)
      .to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &       mean &        std &       min &       25\% &        50\% &        75\% &         max \\
\midrule
price     &     0.1184 &     0.0279 &    0.0766 &    0.0987 &     0.1110 &     0.1287 &      0.2090 \\
load      &  2454.9693 &  2380.2440 &  144.0000 &  790.5000 &  1869.5000 &  3315.0000 &  18621.0000 \\
coal\_cost &    46.0042 &    17.6834 &   24.0381 &   33.4298 &    41.3966 &    51.8319 &    107.4893 \\
CDD       &    64.6627 &   134.9652 &    0.0000 &    0.0000 &     0.0000 &    47.2500 &    761.0000 \\
HDD       &   443.1993 &   412.2670 &    0.0000 &   48.0000 &   367.0000 &   754.5000 &   1794.0000 \\
\bottomrule
\end{tabular}

